In [1]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp500.csv')
#stocks

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
symbol = 'AMZN'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
#type(data)
#print(data) # Response = 200 means success
#print(data.status_code == 200)

In [5]:
print(data)

{'symbol': 'AMZN', 'companyName': 'Amazon.com Inc.', 'primaryExchange': 'C AEQLA)OMSS(TDAAGGLKERSL/EN  NBT', 'calculationPrice': 'tops', 'open': None, 'openTime': None, 'openSource': 'offcilai', 'close': None, 'closeTime': None, 'closeSource': 'iolaicff', 'high': None, 'highTime': 1669470889695, 'highSource': 'aElI tXe ecr empiir', 'low': None, 'lowTime': 1685304657102, 'lowSource': ' iIXmpeitrEel  care', 'latestPrice': 3323.39, 'latestSource': 'IEX real time price', 'latestTime': '2:04:01 PM', 'latestUpdate': 1649988482510, 'latestVolume': None, 'iexRealtimePrice': 3416.2, 'iexRealtimeSize': 6, 'iexLastUpdated': 1626885318322, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 3478.61, 'previousVolume': 2246182, 'change': -53.68, 'changePercent': -0.01589, 'volume': None, 'iexMarketPercent': 0.043234335487982

In [6]:
price = data['latestPrice']
#print(price)
market_cap = data['marketCap']
#print(market_cap/1000000000000)

In [7]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
#final_dataframe = pd.DataFrame([[0,0,0,0]],columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [8]:
final_dataframe.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
    index=my_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AMZN,3323.39,1677519977953,N/A


Looping through the tickers in the list

In [9]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker'][:10]:
#for stock in stocks['Ticker']:
    #print(stock)
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],
            index=my_columns),
        ignore_index=True
        )

In [10]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,138.960,None,N/A
1,AAL,21.000,13608463633,N/A
2,AAP,199.080,None,N/A
3,AAPL,134.820,2226764011157,N/A
4,ABBV,115.170,None,N/A
5,ABC,120.540,None,N/A
6,ABMD,340.290,16047314137,N/A
7,ABT,125.287,None,N/A
8,ACN,302.340,None,N/A
9,ADBE,517.100,248037092005,N/A


Using the Batch API Calls to improve the performance. <br>
IEX Cloud limits the batch API calls to 100 tickers per request.

In [11]:
def chunks(lst, n):
    # Yield successive n-sized chunks from lst.
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [12]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
#symbol_groups
symbol_strings = []
for i in range(0, len(symbol_groups)):
    #print(i)
    #print(symbol_groups[i])
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)
#final_dataframe
for symbol_string in symbol_strings:
    #print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,140.13,None,N/A
1,AAL,20.90,13714325526,N/A
2,AAP,207.16,None,N/A
3,AAPL,134.45,2276857988035,N/A
4,ABBV,112.32,None,N/A
...,...,...,...,...
500,YUM,119.77,None,N/A
501,ZBH,182.39,None,N/A
502,ZBRA,492.03,26532209396,N/A
503,ZION,53.23,8989654664,N/A


In [14]:
portfolio_size = input('Enter the value of your Portfolio: ')

try:
    value = float(portfolio_size)
    #print(val) 
except ValueError:
    print("That's not a number! \nPlease try again:")
    portfolio_size = input('Enter the value of your Portfolio: ')
    value = float(portfolio_size)


That's not a number! 
Please try again:


In [22]:
position_size = value/len(final_dataframe.index)
#print(position_size)
#number_of_apple_shares = position_size/500
#print(math.floor(number_of_apple_shares))

for i in range(0, len(final_dataframe.index)):
    #print(final_dataframe)
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])

final_dataframe


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,140.13,None,4
1,AAL,20.90,13714325526,28
2,AAP,207.16,None,2
3,AAPL,134.45,2276857988035,4
4,ABBV,112.32,None,5
...,...,...,...,...
500,YUM,119.77,None,4
501,ZBH,182.39,None,3
502,ZBRA,492.03,26532209396,1
503,ZION,53.23,8989654664,11


Format the Excel Output

In [29]:
writer = pd.ExcelWriter('recommended.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'recommended', index=False)

In [34]:
background_color = '#0a0a23'
font_color = '#ffffff'
string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [35]:
# writer.sheets['recommended'].set_column('A:A', 18, string_format)
# writer.sheets['recommended'].set_column('B:B', 18, string_format)
# writer.sheets['recommended'].set_column('C:C', 18, string_format)
# writer.sheets['recommended'].set_column('D:D', 18, string_format)
# writer.save()

In [37]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['recommended'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['recommended'].write(f'{column}1', column_formats[column][0], string_format) 

Save the Excel Output

In [38]:
writer.save()

C:\Python38\lib\site-packages\xlsxwriter\workbook.py:336: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
